In [1]:
!pip install mne

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install --pre torch torchvision -f https://download.pytorch.org/whl/nightly/cu111/torch_nightly.html -U

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.pytorch.org/whl/nightly/cu111/torch_nightly.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 MB 7.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 42.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 14.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 10.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 48.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 61.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1
    Uninstalling torchvision-0.13.1:
      Successfully uninsta

In [3]:
######################################################################################################
#  ________   _________   ___    ___ ___  ___  ________  ________  ___  __      _______   _______
# |\   ___  \|\___   ___\|\  \  /  /|\  \|\  \|\   __  \|\   ____\|\  \|\  \   /  ___  \ /  ___  \
# \ \  \\ \  \|___ \  \_|\ \  \/  / | \  \\\  \ \  \|\  \ \  \___|\ \  \/  /|_/__/|_/  //__/|_/  /|
#  \ \  \\ \  \   \ \  \  \ \    / / \ \   __  \ \   __  \ \  \    \ \   ___  \__|//  / /__|//  / /
#   \ \  \\ \  \   \ \  \  /     \/   \ \  \ \  \ \  \ \  \ \  \____\ \  \\ \  \  /  /_/__  /  /_/__
#    \ \__\\ \__\   \ \__\/  /\   \    \ \__\ \__\ \__\ \__\ \_______\ \__\\ \__\|\________\\________\
#     \|__| \|__|    \|__/__/ /\ __\    \|__|\|__|\|__|\|__|\|_______|\|__| \|__| \|_______|\|_______|
#                        |__|/ \|__|
######################################################################################################
#
# Data exploration and example file for submission in the NTX Hackathon challenge
#
######################################################################################################

import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import random

from torch.utils.data import DataLoader
from sklearn.pipeline import make_pipeline
from mne.decoding import Vectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge

In [4]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda:0


In [5]:
# Path to training data
train_path = "/home/deepak/learning_project/student/BrainAge/training/"

In [6]:
# Path to testing data (public test set)
test_path = "/home/deepak/learning_project/student/BrainAge/testing_flat/"

In [7]:
# download data https://filesender.renater.fr/?s=download&token=e1de0ec4-09bc-4194-b85b-59830cb04af3
# download test data from https://codalab.lisn.upsaclay.fr/competitions/8336

condition_ec = "EC"  # use only closed eyes condition for demonstration purpose
condition_eo = "EO"  # use only closed eyes condition for demonstration purpose
train_subj = 1200  # use 10 instead of 1200 training subjects, for demonstration purpose
test_subj = 400  # use 10 instead of 400 testing subjects, for demonstration purpose

In [8]:
raw = mne.io.read_raw(train_path + f"subj{5:04}_{condition_eo}_raw.fif.gz", preload=True)
data = raw.get_data()
print(data.shape)


Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0005_EO_raw.fif.gz...
    Range : 11 ... 9997 =      0.022 ...    19.994 secs
Ready.
Reading 0 ... 9986  =      0.000 ...    19.972 secs...
(129, 9987)


In [9]:
def read_data(path, subj, condition):
  points = 10000
  if condition == condition_ec:
    points = 20000
  data = torch.zeros(subj, 129, points).to(DEVICE)
  for s in range(1, subj + 1):
    fname = f"subj{s:04}_{condition}_raw.fif.gz"
    raw = mne.io.read_raw(train_path + fname, preload=True)
    d = raw.get_data()
    ft = d.shape[-1]
    data[s-1, :, :ft] = torch.tensor(d)
  return data

In [11]:
train_data_eo = torch.tensor(read_data(train_path, train_subj//10, condition_eo), device=DEVICE)
train_data_ec = torch.tensor(read_data(train_path, train_subj//10, condition_ec), device=DEVICE)

Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0001_EO_raw.fif.gz...
    Range : 29 ... 9997 =      0.058 ...    19.994 secs
Ready.
Reading 0 ... 9968  =      0.000 ...    19.936 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0002_EO_raw.fif.gz...
    Range : 18 ... 9997 =      0.036 ...    19.994 secs
Ready.
Reading 0 ... 9979  =      0.000 ...    19.958 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0003_EO_raw.fif.gz...
    Range : 10 ... 9997 =      0.020 ...    19.994 secs
Ready.
Reading 0 ... 9987  =      0.000 ...    19.974 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0004_EO_raw.fif.gz...
    Range : 11 ... 9997 =      0.022 ...    19.994 secs
Ready.
Reading 0 ... 9986  =      0.000 ...    19.972 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0005_EO_raw.fif.gz...
    Rang

/tmp/ipykernel_14417/556691490.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data_eo = torch.tensor(read_data(train_path, train_subj//10, condition_eo), device=DEVICE)


Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0001_EC_raw.fif.gz...
    Range : 15 ... 19997 =      0.030 ...    39.994 secs
Ready.
Reading 0 ... 19982  =      0.000 ...    39.964 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0002_EC_raw.fif.gz...
    Range : 10 ... 19997 =      0.020 ...    39.994 secs
Ready.
Reading 0 ... 19987  =      0.000 ...    39.974 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0003_EC_raw.fif.gz...
    Range : 21 ... 19997 =      0.042 ...    39.994 secs
Ready.
Reading 0 ... 19976  =      0.000 ...    39.952 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0004_EC_raw.fif.gz...
    Range : 32 ... 19997 =      0.064 ...    39.994 secs
Ready.
Reading 0 ... 19965  =      0.000 ...    39.930 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0005_EC_raw.fif.gz...


/tmp/ipykernel_14417/556691490.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data_ec = torch.tensor(read_data(train_path, train_subj//10, condition_ec), device=DEVICE)


In [12]:
train_data_eo.shape

torch.Size([120, 129, 10000])

In [13]:
test_data_eo = torch.tensor(read_data(test_path, test_subj, condition_eo))
test_data_ec = torch.tensor(read_data(test_path, test_subj, condition_ec))

Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0001_EO_raw.fif.gz...
    Range : 29 ... 9997 =      0.058 ...    19.994 secs
Ready.
Reading 0 ... 9968  =      0.000 ...    19.936 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0002_EO_raw.fif.gz...
    Range : 18 ... 9997 =      0.036 ...    19.994 secs
Ready.
Reading 0 ... 9979  =      0.000 ...    19.958 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0003_EO_raw.fif.gz...
    Range : 10 ... 9997 =      0.020 ...    19.994 secs
Ready.
Reading 0 ... 9987  =      0.000 ...    19.974 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0004_EO_raw.fif.gz...
    Range : 11 ... 9997 =      0.022 ...    19.994 secs
Ready.
Reading 0 ... 9986  =      0.000 ...    19.972 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0005_EO_raw.fif.gz...
    Rang

/tmp/ipykernel_14417/2033426799.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_data_eo = torch.tensor(read_data(test_path, test_subj, condition_eo))


Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0001_EC_raw.fif.gz...
    Range : 15 ... 19997 =      0.030 ...    39.994 secs
Ready.
Reading 0 ... 19982  =      0.000 ...    39.964 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0002_EC_raw.fif.gz...
    Range : 10 ... 19997 =      0.020 ...    39.994 secs
Ready.
Reading 0 ... 19987  =      0.000 ...    39.974 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0003_EC_raw.fif.gz...
    Range : 21 ... 19997 =      0.042 ...    39.994 secs
Ready.
Reading 0 ... 19976  =      0.000 ...    39.952 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0004_EC_raw.fif.gz...
    Range : 32 ... 19997 =      0.064 ...    39.994 secs
Ready.
Reading 0 ... 19965  =      0.000 ...    39.930 secs...
Opening raw data file /home/deepak/learning_project/student/BrainAge/training/subj0005_EC_raw.fif.gz...


/tmp/ipykernel_14417/2033426799.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_data_ec = torch.tensor(read_data(test_path, test_subj, condition_ec))


In [14]:
train_data = torch.cat((train_data_eo, train_data_ec), -1)
test_data = torch.cat((test_data_eo, test_data_ec), -1)

In [15]:
# get the age to predict from the CSV file
meta = pd.read_csv(train_path + "train_subjects.csv")
y_train = []
for age in meta["age"][:train_subj]:
    y_train.append(age)

In [16]:
# DataLoader
batch_size = 8

train_loader = DataLoader(
    zip(train_data, y_train),
    batch_size=batch_size,
    num_workers=2
)

train_loader = DataLoader(
    zip(test_data, y_train),
    batch_size=batch_size,
    num_workers=2
)

In [18]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = torch.device(DEVICE)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=3870000, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=1024, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=1024, out_features=1, bias=True)
        )
    def forward(self, x):
        return self.classifier(x)

In [ ]:
def train(model, device, train_loader, batch_size, lr, epochs):
    loss_log = []
    model.train()
    lossfunc = nn.L1Loss()
    optimizer = torch.optim.NAdam(model.parameters(), lr=lr)
    z = torch.zeros(batch_size, 3, 224, 224).to(DEVICE)
    for epoch in range(epochs):
    running_loss = 0.0
    print(f"Epoch #{1 + epoch}")
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        if labels.size(0) == batch_size:
            x = z + images
        else:
            x = torch.zeros(labels.size(0), 3, 224, 224).to(DEVICE) + images

        optimizer.zero_grad()
        output = model(x)
        loss = lossfunc(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    loss_log.append(running_loss)
    plt.plot(loss_log)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('Loss')



---
**THE END**


In [ ]:
train_raws[0].info

In [ ]:
train_raws[0]._data.shape

In [ ]:
fig, ax = plt.subplots(figsize=[15, 5])
start_time = 15
end_time = 16

ax.plot(train_raws[0].get_data(picks='E119', tmin=start_time, tmax=end_time).T)
plt.show()

In [ ]:
# Visualisation of the sensor position in 3D
train_raws[0].plot_sensors(show_names=True, kind="3d")
# Filtering the signal and plotting all 128 channels
train_raws[0].notch_filter([60.0, 120.0, 180.0]).filter(l_freq=1, h_freq=30)
train_raws[0].plot(duration=5, n_channels=129, color={"eeg": "darkblue"})
# Marking some channels as bad (flat electrodes, disconnection, noisy signal, ...)
train_raws[0].info["bads"] = [
    "E7",
    "E48",
    "E106",
    "E112",
    "E55",
    "E31",
    "E105",
    "Cz",
    "E80",
    "E81",
    "E88",
]
# Plotting power spectrum for all electrodes but the bads.
train_raws[0].plot_psd(fmax=50)


In [ ]:

# Get ndarray from MNE raw files to generate train and test input
X_train, X_test = [], []
crop_start, crop_end = 5, 15  # use only a 10s window, from 5s to 15s
for r in train_raws:
    X_train.append(r.copy().crop(tmin=crop_start, tmax=crop_end).get_data())
for r in test_raws:
    X_test.append(r.copy().crop(tmin=crop_start, tmax=crop_end).get_data())

In [ ]:
y_train

In [ ]:
# Create sklearn pipeline, fit and predict
ppl = make_pipeline(Vectorizer(), PCA(n_components=6), Ridge(alpha=0.5))
ppl.fit(X_train, y_train)
y_pred = ppl.predict(X_test)

In [ ]:
# create submission file
dummy_submission = []
for subj, pred in zip(range(1201, 1201 + test_subj), y_pred):
    dummy_submission.append({"id": subj, "age": pred})
pd.DataFrame(dummy_submission).to_csv("mysubmission.csv", index=False)

# zip the csv file (without anything else) and submit it on the website!
